In [ ]:
import os, sys
sys.path.append('..')
os.chdir('..')
import json
from collections import defaultdict
from tasks import *
import networkx as nx 
import numpy as np

In [ ]:
path = 'final_results/np_easy'  # 'np_hard' or 'p_easy' or 'p_hard'
difficulty = path.split('_')[-1]
files = os.listdir(path)
merged_responses = {}
problem_num = 500
dataset_loc = 'dataset'

for f in files:
    if len(f.split('_')) < 2:
        continue
    llm, task = f.split('_')[0], f.split('_')[1]
    with open(f'{path}/{f}', 'r') as file:
        response_dict = json.load(file)
    for i in range(0, problem_num):
        if task not in merged_responses:
            merged_responses[task] = defaultdict(dict)
        merged_responses[task][i][llm] = response_dict[str(i)][llm]
task_list = list(merged_responses.keys())

In [ ]:
for method in ['random', 'greedy', 'approximated']:
    score = {}
    task_name = 'GED'
    task= globals()[task_name + '_Task'](dataset_loc)
    task.load_dataset(difficulty)
    score[task_name] = defaultdict(dict)
    for i in range(0, problem_num):
        score[task_name][i]['gt'] = task.problem_set[i]['exact_answer']
        score[task_name][i]['gt'] = task.approx_solver(task.problem_set[i]['graph'], method=method)
        for llm in merged_responses[task_name][i].keys():
            if llm == 'problem':
                continue
            r = merged_responses[task_name][i][llm]
            if r is None:
                r = ''
                print(i, llm, task_name)
            score[task_name][i][llm] = task.check_solution(i, r)
    metrics = defaultdict(dict)
    less_is_better = ['GED', 'TSP', 'MVC', 'Distance']
    results = []
    task = task_name
    model_list = list(score[task][0].keys())
    model_list.remove('gt')
    for model in model_list:
        metrics[task][model] = {'worse': [],'equal':[], 'better':[]}
        for i in range(0, problem_num):
            diff = score[task][i][model] - score[task][i]['gt']
            if task in less_is_better:
                diff = -diff
            worse = 0 > score[task][i][model] or diff < 0
            equal = score[task][i][model] == score[task][i]['gt']
            better = 0 <= score[task][i][model] and diff > 0

            metrics[task][model]['worse'].append(worse)
            metrics[task][model]['equal'].append(equal)
            metrics[task][model]['better'].append(better)
        avg_worse = sum(metrics[task][model]['worse']) / problem_num
        avg_equal = sum(metrics[task][model]['equal']) / problem_num
        avg_better = sum(metrics[task][model]['better']) / problem_num
        results.append((task, model, avg_worse, avg_equal, avg_better))

    sorted_results = defaultdict(list)

    task_results = [result for result in results if result[0] == task]
    sorted_results[task] = sorted(task_results, key=lambda x: x[2], reverse=False)

    # Print sorted results for each task
    for task, task_results in sorted_results.items():
        print(f"\nTask: {task}, method: {method}")
        for result in task_results:
            print(f"Model: {result[1]}, worse: {result[2]:.3f}, equal: {result[3]:.3f}, better: {result[4]:.3f}")